In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
from scipy.spatial import distance
from matplotlib import pyplot as plt
from keras.models import load_model
from PIL import Image
from sklearn.model_selection import train_test_split
import os

In [2]:
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras import Sequential
from keras.layers import Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

In [7]:
train_dir = r'C:\Users\jspc8\Downloads\Behind the Mask\Behind the Mask\train'
test_dir = r'C:\Users\jspc8\Downloads\Behind the Mask\Behind the Mask\test'
#val_dir = '/content/Face Mask Dataset/Validation'

In [8]:
train_datagen = ImageDataGenerator(rescale=1.0/255, horizontal_flip=True, zoom_range=0.2,shear_range=0.2)
train_generator = train_datagen.flow_from_directory(directory=train_dir,target_size=(128,128),class_mode='categorical',batch_size=32)


test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = train_datagen.flow_from_directory(directory=test_dir,target_size=(128,128),class_mode='categorical',batch_size=32)

Found 2646 images belonging to 2 classes.
Found 1146 images belonging to 2 classes.


In [72]:
vgg19 = VGG19(weights='imagenet',include_top=False,input_shape=(128,128,3))

for layer in vgg19.layers:
    layer.trainable = False
    
model = Sequential()
model.add(vgg19)
model.add(Flatten())
model.add(Dense(2,activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 4, 4, 512)         20024384  
                                                                 
 flatten_2 (Flatten)         (None, 8192)              0         
                                                                 
 dense_2 (Dense)             (None, 2)                 16386     
                                                                 
Total params: 20,040,770
Trainable params: 16,386
Non-trainable params: 20,024,384
_________________________________________________________________


In [73]:
from tensorflow import keras;    
from tensorflow.keras import metrics;
model.compile(optimizer="adam",loss="categorical_crossentropy", metrics=['acc', keras.metrics.Precision(), keras.metrics.Recall()])

In [75]:
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=len(train_generator)//32,
                              epochs=15
                              ,validation_data=test_generator,
                              validation_steps=len(test_generator)//32)

C:\Users\jspc8\AppData\Local\Temp/ipykernel_19796/3469203096.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator=train_generator,


Epoch 1/15
2/2 [==============================] - 9s 6s/step - loss: 0.6618 - acc: 0.6406 - precision_2: 0.5833 - recall_2: 0.1094 - val_loss: 0.7219 - val_acc: 0.4688 - val_precision_2: 0.8235 - val_recall_2: 0.4375
Epoch 2/15
2/2 [==============================] - 8s 5s/step - loss: 0.6327 - acc: 0.5938 - precision_2: 0.8800 - recall_2: 0.3438 - val_loss: 0.4116 - val_acc: 0.9062 - val_precision_2: 1.0000 - val_recall_2: 0.3125
Epoch 3/15
2/2 [==============================] - 8s 5s/step - loss: 0.4460 - acc: 0.8125 - precision_2: 1.0000 - recall_2: 0.3281 - val_loss: 0.3717 - val_acc: 0.8125 - val_precision_2: 1.0000 - val_recall_2: 0.5000
Epoch 4/15
2/2 [==============================] - 8s 5s/step - loss: 0.4205 - acc: 0.8281 - precision_2: 1.0000 - recall_2: 0.4688 - val_loss: 0.3406 - val_acc: 0.9375 - val_precision_2: 1.0000 - val_recall_2: 0.2812
Epoch 5/15
2/2 [==============================] - 8s 5s/step - loss: 0.3089 - acc: 0.9375 - precision_2: 0.9688 - recall_2: 0.4844 -

C:\Users\jspc8\AppData\Local\Programs\Python\Python39\lib\site-packages\PIL\Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


2/2 [==============================] - 8s 5s/step - loss: 0.1952 - acc: 0.9531 - precision_2: 1.0000 - recall_2: 0.7188 - val_loss: 0.1584 - val_acc: 0.9688 - val_precision_2: 1.0000 - val_recall_2: 0.7812
Epoch 12/15
2/2 [==============================] - 8s 5s/step - loss: 0.1152 - acc: 0.9688 - precision_2: 0.9825 - recall_2: 0.8750 - val_loss: 0.2159 - val_acc: 0.9375 - val_precision_2: 0.9630 - val_recall_2: 0.8125
Epoch 13/15
2/2 [==============================] - 8s 5s/step - loss: 0.1472 - acc: 0.9531 - precision_2: 1.0000 - recall_2: 0.8125 - val_loss: 0.1375 - val_acc: 0.9375 - val_precision_2: 1.0000 - val_recall_2: 0.7812
Epoch 14/15
2/2 [==============================] - 8s 5s/step - loss: 0.1544 - acc: 0.9375 - precision_2: 0.9808 - recall_2: 0.7969 - val_loss: 0.2399 - val_acc: 0.8750 - val_precision_2: 0.9600 - val_recall_2: 0.7500
Epoch 15/15
2/2 [==============================] - 8s 5s/step - loss: 0.1474 - acc: 0.9531 - precision_2: 0.9804 - recall_2: 0.7812 - val_lo

In [76]:
test_loss, test_acc, test_precision, test_recall = model.evaluate(test_generator, steps=7)

7/7 [==============================] - 18s 2s/step - loss: 0.1869 - acc: 0.9554 - precision_2: 0.9780 - recall_2: 0.7946


In [77]:
model.save('Dl_model.h5')

In [78]:
import tensorflow as tf

In [79]:
new_model = tf.keras.models.load_model('Dl_model.h5')

In [91]:
img = cv2.imread(r'C:\Users\jspc8\Downloads\Behind the Mask\Behind the Mask\sample\1.jpg')

img = cv2.resize(img,(128,128)) 
img = np.reshape(img, [1,128,128,3])/255.0

pred_prob = new_model.predict(img)

pred_prob

array([[0.74828106, 0.18351752]], dtype=float32)

In [92]:
img = cv2.imread(r'C:\Users\jspc8\Downloads\Behind the Mask\Behind the Mask\sample\14.png')
#plt.imshow(img)

img = cv2.resize(img,(128,128)) 
img = np.reshape(img, [1,128,128,3])/255.0

pred_prob = new_model.predict(img)

pred_prob

array([[0.04697245, 0.8383082 ]], dtype=float32)

In [94]:
img = cv2.imread(r'C:\Users\jspc8\Downloads\Behind the Mask\Behind the Mask\sample\34.png')
#plt.imshow(img)

img = cv2.resize(img,(128,128)) 
img = np.reshape(img, [1,128,128,3])/255.0

pred_prob = new_model.predict(img)

pred_prob

array([[0.93253064, 0.02053204]], dtype=float32)

In [95]:
img = cv2.imread(r'C:\Users\jspc8\Downloads\Behind the Mask\Behind the Mask\sample\81.png')
#plt.imshow(img)

img = cv2.resize(img,(128,128)) 
img = np.reshape(img, [1,128,128,3])/255.0

pred_prob = new_model.predict(img)

pred_prob

array([[0.34603882, 0.3204388 ]], dtype=float32)